# 1b. Backpropagation

So I spent the past week and a half convincing myself that I understand the math behind backpropagation, and now I'm going to first prove it all over again here, and then implement it for my baby neural net, so that I can start training it. 

I'm following along with [Nielsen Chapter 2](http://neuralnetworksanddeeplearning.com/chap2.html), which I really can't praise highly enough - if you don't currently have access to an ML professor who can walk you through the math in person, this chapter is a good substitute. Now that I've gone through it several times and written out my own proofs of various parts, I'm going to try to explain/implement this without referring back to the text.

### A brief refresher on gradient descent

The basic idea behind training a neural net is that we start with random weights and biases, and want to move towards values for the weights and biases that give us the best performance on our set of training data. Let's unpack this:
* __training data:__ this is supervised learning; we've got a bunch of input values along with the expected outputs, and we're trying to have the net arrive at some generalized relationship between them that will hold true for new data. In practice, there are a lot different methodologies for how to use your training data, which I'll get into in more detail in the next journal when I actually start training my neural net.
* __performance:__ some measure of how well the net does relative to the expected outputs of the training data -  how frequently is it right (on a classification-type problem), how close is it on average (on an estimation-type problem), etc. Performance on a training set is measured by some cost function that aggregates from performance on each example.
* __best:__ not necessarily 100% accurate (since that's probably overfitting and won't generalize well), but reasonably good, depending on what problem we're dealing with and how accurate we need to be.
* __move towards values for the weights and biases:__ we're trying to minimize the cost function, that is, find the set of weights and biases that produces the minimum cost for our training data. However, there's not a simple closed-form expression that lets us directly calculate the weights and biases that will lead to a global minimum - there are dozens of weight/bias variables involved even in a small neural net. What we can do instead is find the derivative at our current set of weights and biases, in the form of a partial derivative for each weight and bias individually. Then, assuming the cost function is reasonably well-behaved, we can take a small step "downhill" for each variable, which should bring us a bit closer overall to a minimum (and a global minimum if we're very lucky, but that doesn't really matter). The general method at work here is gradient descent; backpropagation is an algorithm for efficiently calculating the necessary gradient for a neural net.

### Notation

To start off:
* A particular layer of a neural net will be denoted as $l$. $l-1$, $l+1$, etc. refer to the layers before or after layer $l$, and $L$ denotes the final (output) layer.
* Assuming we've passed some value through the net, $a_j^l$ is the activation value of the $j$th neuron in layer $l$ (the output of the activation function). $z_j^l$ is the pre-activation value, that is, the weighted sum of the inputs from the previous layer plus the bias (so $a_j^l = \sigma(z_j^l)$ where $\sigma$ is that neuron's activation function. We can also refer to $a^l$, $b^l$, or $z^l$ to denote the vector of the given quantities for layer $l$ (e.g. $a^l = [a_0^l, ..., a_n^l]$, where layer $l$ has $n$ neurons total).
* $w_{jk}^l$ is the current weight from neuron $k$ in layer $l-1$ to neuron $j$ in layer $l$ (i.e. $w_{jk}^l$ is the weight *to* $j$ *from* $k$. (This is backwards for reasons that make more sense when you think of a neural net as a list of weight matrices and bias vectors,  but we'll use this convention to abide by historical precedent.) $b_j^l$ is the current bias for neuron $j$ in layer $l$.

To train a neural net, we have some set of input values that are paired with the corresponding desired output values. We also have a cost function of some kind that provides a measure of the difference between the desired output and the actual output of the neural net.

* If our net has $n$ neurons in the input layer and $m$ in the output layer, then $(x, y)$ is a training example, where $x$ is a vector of length $n$ and $y$ is a vector of length $m$.
* If we feed a training input $x$ through the network, we get some output vector $a^L$.

### The cost function

Our set of training examples is fixed; we're trying to change all the weights and biases in the network so that overall, $a^L$ for each $x$ is pretty close to the corresponding $y$. So at a high level, think of the cost function as something that takes a bunch of candidate values for each weight and bias, applies those weights and biases to each training example, and outputs some measure of the cumulative cost. We want to find some particular set of weights and biases that minimizes the cumulative cost. Our cost function is not a function $C(y, a^L)$ for particular examples; it's more like a function $C_T(w^L, w^{L-1}, ..., w^0, b^L, b^{L-1}, ..., b^0)$, parameterized by the training set $T$ and varying over the weights and biases. This is why it makes sense to talk about minimizing the cost by changing them - it's a function of those quantities and we can do calculus to it.

The overall cost function $C$ measures some cumulative error over a bunch of training examples. For backpropagation, though, we want to be able to see how the net processes individual examples, so we can figure out how the net needs to change to better suit each one. So we want our overall cost function to be easily decomposable into a cost function for a particular example - call this $C_x$. So without explicitly defining our cost function, we're going to impose a few restrictions on it:

* $C_T(w, b) = \frac{\epsilon}{|{T}|}\sum_{(x, y) \in T}C_x(w, b)$, that is, for a particular combo of weights and biases, $C_T$ is the average of individual errors $C_x$ over all pairs in the training set, maybe scaled by some constant $\epsilon$ depending on the exact form.
* $C_x(w, b)$ can be expressed as a function of the final outputs of the network $a_0^L, ..., a_n^L$ (which are themselves functions of the weights and biases).
* $C_x(w, b)$ should be easily differentiable with respect to $a_0^L, ..., a_n^L$. We want to minimize $C_T$, which means we're going to be taking derivatives with respect to each weight and bias, and in this form the derivative of  $C_T$ works out to the (scaled) average of the derivatives of all the individual $C_x$s. (I'm handwaving a bit here by conflating weights/biases with network outputs, but $C_x$ is basically a nasty composition of functions that we'll deconstruct using the chain rule - you can think of it as $C_x(a^L(w, b))$ if it makes this clearer. I promise the math works out.)

Depending on the source, there various names for $C_T$ and $C_x$, e.g. $C_T$ is the cost function, each $C_x$ is a loss function, $C_T$ is the objective function, $C_x$ is an error function, etc. I'm arbitrarily using "cost function" for both $C_T$ and $C_x$ and using the subscript to differentiate between the overall and the particular version.

### Putting it all together

We ultimately want to find the gradient of the overall cost function $\nabla C_T$, which is just shorthand for the vector of partial derivatives $\frac{\partial C_T}{\partial w_{jk}^l}$ and $\frac{\partial C_T}{\partial b_{j}^l}$ for all layers $l$ and connected neurons $j \in l$, $k \in l - 1$ in the network. To do that, we need to calculate the gradient vector $\nabla C_x$ for each example $x \in T$, take the elementwise average of those individual gradients as $\nabla C_T$, and then adjust each weight and bias in the network based on its respective partial derivative: if $\frac{\partial C_T}{\partial w_{jk}^l}$ is positive, then $C_T$ is increasing in tandem with $w_{jk}^l$, so we should decrease $w_{jk}^l$ to move down the slope towards the minimum. The reverse applies when $\frac{\partial C_T}{\partial w_{jk}^l}$ is negative. The exact step size should be proportional to the size of $\frac{\partial C_T}{\partial w_{jk}^l}$ - if the slope is very steep, we can take a larger step and still be confident that we'll hit some lower point; if the slope is very shallow, we want to move more slowly to avoid overshooting the minimum. (As mentioned above, all this assumes that $C_T$ is reasonably well-behaved.)

So backpropagation boils down to something pretty straightforward: how do we efficiently calculate $\nabla C_x$ for a training example $x$?

### Backpropagation in four easy steps

__Goal:__ Expressions for $\frac{\partial C_x}{\partial w_{jk}^l}$ and $\frac{\partial C_x}{\partial b_j^l}$ for all weights $w_{jk}^l$ and biases $b_j^l$.

__Definition:__ Let $\delta_j^l \equiv \frac{\partial C_x}{\partial z_j^l}$, where $z_j^l$ is the pre-activation value of the $j$th neuron in layer $l$ on training input $x$. We can think of the quantity $\delta_j^l$ as the _error_ of that neuron: the larger the quantity, the more we can potentially change the  value of $C_x$ by tweaking $z_j^l$, and vice versa. Let $\delta^l$ be the vector of per-neuron errors for layer $l$, that is, $\delta^l = [\delta_0^l, ..., \delta_n^l]$.

__(1)__ $\delta_j^L = \frac{\partial C_x}{\partial a_j^L}\sigma'(z_j^L)$

__Proof of (1):__ Since $C_x$ actually takes $a_j^L = \sigma(z_j^L)$ as a parameter, we can rewrite this expression using the chain rule:

$$\delta_j^L \equiv \frac{\partial C_x}{\partial z_j^L} = \sum_{k\in L}\frac{\partial C_x}{\partial a_k^L}\frac{\partial a_k^L}{\partial z_j^L}$$

Note that $\frac{\partial a_k^L}{\partial z_j^L} = 0$ for $k \neq j$ (since the activation value at each neuron $a_k^l$ depends only on the pre-activation value at the same neuron $z_k^l$, not on any others). Thus we can write 

$$ \sum_{k\in L}\frac{\partial C_x}{\partial a_k^L}\frac{\partial a_k^L}{\partial z_j^L} = \frac{\partial C_x}{\partial a_j^L} \frac{\partial a_j^L}{\partial z_j^L}$$

$$ = \frac{\partial C_x}{\partial a_j^L}\frac{\mathop{d}}{\mathop{dz_j^L}}(\sigma(z_j^l)) $$

$$ = \frac{\partial C_x}{\partial a_j^L}\sigma'(z_j^l). $$

Per the discussion above, we can safely assume that $\frac{\partial C_x}{\partial a_j^L}$ (the partial derivative of $C_x$ with respect to the $j$th output) is easily stated (e.g. when $C_T$ is the mean squared error cost function, we have $C_x = \frac{1}{2}\sum_{j}(a_j^L - y_j)^2$, so $\frac{\partial C_x}{\partial a_j^L} = a_j^L - y_j$). Similarly, any standard activation function $\sigma$ will have an easily expressed $\sigma'$.

__(2)__ $\delta_j^l = \sigma'(z_j^l) (w^{l+1})^\intercal[j] \cdot \delta^{l+1} $

__Proof of (2):__ As above, we rewrite using the chain rule, this time using $z_k^{l+1}$ as the intermediate variable.

$$\delta_j^l \equiv \frac{\partial C_x}{\partial z_j^l} = \sum_{k \in l + 1}\frac{\partial C_x}{\partial z_k^{l + 1}}\frac{\partial z_k^{l+1}}{\partial z_j^l}$$

$$ = \sum_{k \in l + 1}\delta_k^{l + 1}\frac{\partial z_k^{l+1}}{\partial z_j^l} \text{ (by definition)} $$

$$ = \sum_{k \in l + 1}\delta_k^{l + 1}\frac{\mathop{d}}{\mathop{dz_j^l}}[\sum_{i \in l}w_{ki}^{l+1}\sigma(z_i^l) + b_k^{l+1}],$$

explicitly expanding the quantity $z_k^{l+1}$ as the weighted sum of inputs to neuron $k$ in layer $l+1$ from the neurons in layer $l$ (indexed by $i$) plus the bias $b_k^{l+1}$.

Taking the derivative of this sum with respect to $z_j^l$ yields

$$ = \sum_{k \in l + 1}\delta_k^{l + 1}(w_{kj}^{l+1}\sigma'(z_j^l)) $$

$$ = \sigma'(z_j^l)\sum_{k \in l + 1}w_{kj}^{l+1}\delta_k^{l + 1} $$

$$ = \sigma'(z_j^l) (w^{l+1})^\intercal[j] \cdot \delta^{l+1},$$

where $(w^{l+1})^\intercal[j]$ denotes the $j$th row of the transposed weight matrix, that is, a row vector $[w_0j^{l+1}, ..., w_nj^{l+1}]$ containing the weights from neuron $j$ in layer $l$ to each of the $n$ neurons in layer $l + 1$, that is, we're just rewriting the summation in the previous line as a dot product of two length-$n$ vectors.

We can get a more elegant expression by generalizing over $j$ and instead writing

$$ \delta^l = (w^{l+1})^\intercal \delta^{l+1} \times \sigma'(z^l), $$

where $\times$ indicates elementwise multiplication and $\sigma'(z^l) = [\sigma(z_0^l), ..., \sigma(z_m^l)]$.

By taking (1) as our base case, we can iteratively take (2) to find $\delta^l$ for all layers $l$.

__(3)__ $\frac{\partial C_x}{\partial b_j^l} = \delta_j^l$

__Proof of (3):__ Expand using the chain rule again to get a $\delta$-term, and simplify the remainder:

$$ \frac{\partial C_x}{\partial b_j^l} = \sum_{k \in l}\frac{\partial C_x}{\partial z_k^l} \frac{\partial z_k^l}{\partial b_j^l}$$

$$ = \sum_{k \in l}\delta_k^l \frac{\partial z_k^l}{\partial b_j^l}$$

$$ = \delta_j^l \frac{\partial z_j^l}{\partial b_j^l},$$

since $z_k^l$ is only affected by $b_j^l$ when $k = j$ (the bias at neuron $j$ does not factor into any of the other neurons in the layer), so all other terms in this sum go to zero. Continuing, expand quantity $z_j^l$ to get

$$ = \delta_j^l \frac{\mathop{d}}{\mathop{db_j^l}}(w^l[j] \cdot a^{l-1}) + b_j^l) $$

$$ = \delta_j^l.$$

This is basically a consequence of the fact that the bias is just a constant term, so changing the bias by some quantity has exactly the same impact as changing the pre-activation quantity $z$ by the same amount after calculating it, so the derivative is the same with respect to either one.

__(4)__ $\frac{\partial C_x}{\partial w_{jk}^l} = a_k^{l-1} \delta_j^l$

__Proof of (4):__ Chain rule again.

$$ \frac{\partial C_x}{\partial w_{jk}^l} = \sum_{i \in l} \frac{\partial C_x}{\partial z_i^l} \frac{\partial z_i^l}{\partial w_{jk}^l} $$

$$ = \frac{\partial C_x}{\partial z_j^l} \frac{\partial z_j^l}{\partial w_{jk}^l}, $$

again eliminating most of the summation by noting that the weight to $k$ in $l$ from $j$ in $l-1$ only affects $z_i^l$ when $i=j$. Continuing,

$$ = \delta_j^l \frac{\mathop{d}}{\mathop{dw_{jk}^l}}(w_{j0}^l a_0^{l-1} + ... + w_{jk}^l a_k^{l-1} + ... + w_{jn}^l a_n^{l-1} + b_j^l) $$

$$ = \delta_j^l a_k^{l-1}. $$

### So:

Every time we pass an input $x$ forward through our network, we calculate $z^l$ and $a^l$ for each layer. Then we can use (1) and (2) to propagate error backwards to find $\delta^l$ for each layer, and then in turn use (3) and (4) to calculate $\frac{\partial C_x}{\partial w_jk^l}$ and $\frac{\partial C_x}{\partial b_j^l}$ for each weight in bias in terms of the associated errors. This gives us the desired gradient $\nabla C_x$. Then we just do this for a bunch of different $x$s and take the elementwise average of all the resulting  $\nabla C_x$s to get $\nabla C_T$. Then we tweak the weights and biases based on that and start the whole process again until we decide to stop. That's it. The individual calculations end up being pretty simple, which kinda makes sense when you consider that the net is basically a nest of linear combinations (excluding the activation functions, but again, those derivatives are pretty straightforward.)

### Implementation v1

I've sort of kneecapped myself here by modeling my net as a graph with weighted edges instead of a set of matrices and bias vectors. In the interest of, I guess, masochism, I'm gonna try and implement the world's slowest backpropagation process. Putting the existing code below so that each notebook is self-contained. One issue with the original code is that there's no simple way to access the derivative of the activation function for a given neuron, so we'll bundle both functions together in a new wrapper and tweak the Neuron class slightly to reflect the new definition. We'll also add an additional field to each Neuron to track pre-activation value separately from activation value.

In [20]:
import math

class Activation(object):
    def __init__(self, f, f_prime):
        self.f = f
        self.f_prime = f_prime

    def activate(self, x):
        return self.f(x)

    def deriv(self, x):
        return self.f_prime(x)

def identity_f(x):
    return x
def identity_fprime(x):
    return 1
identity = Activation(identity_f, identity_fprime)

def sigmoid_f(x):
    return 1 / (1 + math.e ** (-x))
def sigmoid_fprime(x):
    return sigmoid_f(x) * (1 - sigmoid_f(x))
sigmoid = Activation(sigmoid_f, sigmoid_fprime)

activation_types = {
    "identity": identity,
    "sigmoid": sigmoid,
}

class Neuron(object):
    
    # Initialize with an activation function
    # Bias, weights, and links will be determined later
    def __init__(self, f, b):
        self.activation = f
        self.bias = b
        self.upstream_neurons = []
        self.upstream_weights = []
        self.downstream_neurons = []
        self.downstream_weights = []
        self.pre_activation_value = None
        self.activation_value = None

    # Link from an earlier neuron
    # Input array and weight array are paired
    def link_input(self, input_neuron, weight):
        self.upstream_neurons.append(input_neuron)
        self.upstream_weights.append(weight)
        input_neuron.downstream_neurons.append(self)
        input_neuron.downstream_weights.append(weight)

    # Link to a later neuron
    # Should use one or the other of the link functions for any given net
    # to avoid double binding neurons
    def link_output(self, output_neuron, weight):
        self.downstream_neurons.append(output_neuron)
        self.downstream_weights.append(weight)
        output_neuron.upstream_neurons.append(self)
        output_neuron.upstream_weights.append(weight)
        
    def set_bias(self, b):
        self.bias = b
        
    def set_initial_value(self, v):
        self.activation_value = v
        
    def activate(self):
        total = 0
        for i in range(len(self.upstream_neurons)):
            neuron = self.upstream_neurons[i]
            weight = self.upstream_weights[i]
            total += neuron.activation_value * weight
        total += self.bias
        self.pre_activation_value = total
        total = self.activation.activate(total)
        self.activation_value = total


class NeuralNet(object):
    
    def __init__(self):
        self.layers = []
        self.input_size = 0
        self.output_size = 0
        self.n_layers = 0

    def add_layer(self, n, f):
        self.layers.append([Neuron(f, 0) for _ in range(n)])

    def fully_connect_layers(self, l1, l2, weights):
        for i in range(len(l1)):
            for j in range(len(l2)):
                input_neuron = l1[i]
                output_neuron = l2[j]
                weight = weights[j][i]
                input_neuron.link_output(output_neuron, weight)

    # Spec of the format [("identity|sigmoid|tanh|relu", n)]
    # Initialize a neural net with all weights 1 and all biases 0 
    def build_from_spec(self, spec):
        for (activation, n) in spec:
            if activation not in activation_types:
                print(f"Unrecognized activation function: {activation}")
                return
            ac = activation_types[activation]
            self.add_layer(n, ac)
        for i in range(len(self.layers) - 1):
            l1 = self.layers[i]
            l2 = self.layers[i + 1]
            n = len(l1)
            m = len(l2)
            weights = [[1 for _ in range(n)] for _ in  range(m)]
            self.fully_connect_layers(l1, l2, weights)
        self.input_size = len(self.layers[0])
        self.output_size = len(self.layers[-1])
        self.n_layers =  len(self.layers)


    # input_vector is just a list-like object that can be indexed
    def passthrough(self, input_vector):
        if len(input_vector) != self.input_size:
            print(f"Mismatched input: expected size {self.input_size} but got size {len(input_vector)}")
        for i in range(self.input_size):
            self.layers[0][i].set_initial_value(input_vector[i])
        for layer in self.layers[1:]:
            for neuron in layer:
                neuron.activate()
        output_vector = [neuron.activation_value for neuron in self.layers[-1]]
        return output_vector


I'm planning to use the classic iris dataset to train and validate my network implementation - if you haven't encountered it, it looks like this:

In [16]:
f = open("../datasets/iris.csv")
lines = f.readlines()
f.close()

for line in lines[:6]:
    print(line)

sepallength,sepalwidth,petallength,petalwidth,class

5.1,3.5,1.4,0.2,Iris-setosa

4.9,3.0,1.4,0.2,Iris-setosa

4.7,3.2,1.3,0.2,Iris-setosa

4.6,3.1,1.5,0.2,Iris-setosa

5.0,3.6,1.4,0.2,Iris-setosa



So 4 features and 3 categories, which determines the input and output layers for our network. Let's start small for now, with a single hidden layer of 5 neurons (chosen pretty much randomly). Realistically the final layer should be a softmax or something but for now we'll leave it as a sigmoid and plan on hooking up some extra machinery that tells us what the highest prediction was.

In [22]:
iris_net = NeuralNet()
iris_net.build_from_spec([("identity", 4), ("sigmoid", 5), ("sigmoid", 3)])